In [ ]:
from get_features import *
from model import *

In [ ]:
feature = np.load('../truncated_feature_normalized.npy')
target = np.load('../truncated_output.npy')
seqlen = np.load('../seqlen.npy')

In [ ]:
feature.shape

In [ ]:
clusters = np.unique(feature[:,:,:,:,0])
n_clusters = clusters.shape[0]

In [ ]:
HIDDEN_SIZE = 128

In [ ]:
INPUT_SIZE = feature.shape[-1]-1

In [ ]:
train_idx = np.random.choice(np.arange(250), replace=False, size=150)
train_bool = np.zeros(250).astype(bool)
train_bool[train_idx] = True

val_idx = np.random.choice(np.arange(250)[~train_bool], replace=False, size=50)
val_bool = np.zeros(250).astype(bool)
val_bool[val_idx] = True

test_bool = ~np.logical_or(train_bool, val_bool)

In [ ]:
# # save indices for reproducibility
# np.save('train_bool_newnewnew.npy', train_bool)
# np.save('val_bool_newnewnew.npy', val_bool)
# np.save('test_bool_newnewnew.npy', test_bool)

In [ ]:
train = feature[:,:,train_bool].reshape((-1,feature.shape[-2],feature.shape[-1]))
train = torch.from_numpy(train).cuda().float()
print(train.shape)
val = feature[:,:,val_bool].reshape((-1,feature.shape[-2],feature.shape[-1]))
val = torch.from_numpy(val).cuda().float()
print(val.shape)
test = feature[:,:,test_bool].reshape((-1,feature.shape[-2],feature.shape[-1]))
test = torch.from_numpy(test).cuda().float()
print(test.shape)

In [ ]:
train_target = target[:,:,train_bool].reshape((-1,feature.shape[-2]))
train_target = torch.from_numpy(train_target).cuda().float()
print(train_target.shape)
val_target = target[:,:,val_bool].reshape((-1,feature.shape[-2]))
val_target = torch.from_numpy(val_target).cuda().float()
print(val_target.shape)
test_target = target[:,:,test_bool].reshape((-1,feature.shape[-2]))
test_target = torch.from_numpy(test_target).cuda().float()
print(test_target.shape)

In [ ]:
train_seq_len = np.tile(seqlen[:,None], (1,feature.shape[1],1))[:,:,train_bool].reshape(-1)
print(train_seq_len.shape)
val_seq_len = np.tile(seqlen[:,None], (1,feature.shape[1],1))[:,:,val_bool].reshape(-1)
print(val_seq_len.shape)
test_seq_len = np.tile(seqlen[:,None], (1,feature.shape[1],1))[:,:,test_bool].reshape(-1)
print(test_seq_len.shape)

In [ ]:
model = define_model(n_clusters=n_clusters, input_size=INPUT_SIZE, output_size=1, hidden_dim=HIDDEN_SIZE, n_layers=3, total_length = feature.shape[-2])

In [ ]:
lr=0.01

In [ ]:
best_epoch = do_train(model, train, train_seq_len, train_target, val, val_seq_len, val_target, lr=lr)

In [ ]:
model.load_state_dict(torch.load('./state_dict_epoch={}_lr={}.pt'.format(16, lr)))
model.eval()

In [ ]:
covariates = ['lab','xyz','reward', 'wheel', 'contrast', 'lick','max_ptp', 'wf_width']

In [ ]:
effect_sizes = dict()
n_samples = 500
test_feature = test.reshape(feature[:,:,test_bool].shape)
for i in range(feature.shape[0]):
    for j in notebook.tqdm(range(feature.shape[1])):
        effect_size = get_effect_sizes(model, test_feature, test_seq_len, covariates, i, j, n_samples)
        effect_sizes[(i,j)] = effect_size